In [1]:
import random 
import numpy as np
import pandas as pd
from collections import Counter
from sklearn import preprocessing
from collections import defaultdict

In [11]:
k = 7
# data = 'wine_data.csv'
data = 'glass.csv'
def get_data(data):
    X = pd.read_csv(data, sep=",", header=None)
    print(X.head())
    X = X.drop(0, axis=1)
    print(X.head())
    X_class_col = X[10]
#     maxmin_scalar = preprocessing.MinMaxScaler().fit_transform(X)
    maxmin_scalar = preprocessing.MinMaxScaler().fit_transform(X.ix[:,:10])
#     print(maxmin_scalar)
    X = pd.DataFrame(maxmin_scalar)
    X = pd.concat([X, X_class_col], axis=1)
    X = X.values.tolist()
#     print(X.head())
    return X
X = get_data(data)
print(X[0])


   0        1      2     3     4      5     6     7    8    9   10
0   1  1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.0  0.0   1
1   2  1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.0  0.0   1
2   3  1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.0  0.0   1
3   4  1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.0  0.0   1
4   5  1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.0  0.0   1
        1      2     3     4      5     6     7    8    9   10
0  1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.0  0.0   1
1  1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.0  0.0   1
2  1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.0  0.0   1
3  1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.0  0.0   1
4  1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.0  0.0   1
[0.4328358208955194, 0.437593984962406, 1.0, 0.25233644859813087, 0.3517857142857146, 0.00966183574879227, 0.30855018587360594, 0.0, 0.0, 0.0, 1.0]


In [86]:
lst = [[2,3,4], [4,5,6]]
np_lst = np.array(lst)
print(pd.DataFrame(lst))
array = np.mean(lst, axis=0)
print(type(array))
print(np_lst[:, -1])
# Counter(array[0]).most_common(1)[0][0]

   0  1  2
0  2  3  4
1  4  5  6
<type 'numpy.ndarray'>
[4 6]


In [12]:
def is_converged(centroids, old_centroids):
    return set([tuple(a) for a in centroids]) == set([tuple(b) for b in old_centroids])

def get_distance(x, c):
    return np.linalg.norm(np.array(x)-np.array(c))

def get_clusters(X, centroids):
    clusters = defaultdict(list)
    for x in X:
        # cluster is a num to indicate the # of centroids
        cluster = np.argsort([get_distance(x[:-1], c[:-1]) for c in centroids])[0]
        clusters[cluster].append(x)
    return clusters

def get_centeroids(old_centroids, clusters):
    new_centroids = []
    keys = sorted(clusters.keys())
    for k in keys:
        new_centroid = np.mean(clusters[k], axis=0)
        new_centroid[len(clusters[0][0])-1] = Counter([clusters[k][i][-1] for i in range(len(clusters[k]))]).most_common(1)[0][0]
        new_centroids.append(new_centroid)
    return new_centroids

def find_centers(X, K):
    old_centroids = random.sample(X, K)
    centroids = random.sample(X, K)
    iteration = 0
    while not is_converged(centroids, old_centroids):
        old_centroids = centroids
        clusters = get_clusters(X, centroids)
        centroids = get_centeroids(old_centroids, clusters)
        iteration += 1
    return (centroids, clusters, iteration)

In [13]:
X = get_data(data)
num_instances = len(X)
centroids, clusters , iteration= find_centers(X, k)
print centroids, iteration

   0        1      2     3     4      5     6     7    8    9   10
0   1  1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.0  0.0   1
1   2  1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.0  0.0   1
2   3  1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.0  0.0   1
3   4  1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.0  0.0   1
4   5  1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.0  0.0   1
        1      2     3     4      5     6     7    8    9   10
0  1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.0  0.0   1
1  1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.0  0.0   1
2  1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.0  0.0   1
3  1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.0  0.0   1
4  1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.0  0.0   1
[array([ 0.30177672,  0.35314602,  0.75553862,  0.31128054,  0.51968985,
        0.08394779,  0.30656916,  0.00843776,  0.4375645 ,  0.13157895,  2.        ]), array([  2.53523825e-01,   3.71455913e-01,   7.83721401e-01,

In [14]:
def get_purity(clusters, centroids, num_instances):
    counts = 0
    for k in clusters.keys():
        labels = np.array(clusters[k])[:, -1]
        counts += Counter(labels).most_common(1)[0][1]
    return float(counts)/num_instances

In [15]:
purity = get_purity(clusters, centroids, num_instances)
print(purity)
# for k in clusters.keys():
#         points = np.array(clusters[k])
#         class_attr = Counter(points[:, -1]).most_common(1)
#         print class_attr
# print '\n'

0.742990654206


In [16]:
0.95
0.74

0.74